<a href="https://colab.research.google.com/github/cras-lab/BlockChain_API/blob/main/Ethereum_TX_n_GAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

web3모듈은 파이썬을 사용하여 이더리움 블록체인과 상호작용하는 라이브러리<BR>
이더리움 블록에서 평균 트랜잭션 개수와 개스비용을 읽어오는 예제

In [21]:
pip install -q web3

필요한 모듈을 임포트한다.

In [22]:
from web3 import Web3
import matplotlib.pyplot as plt

Infura의 키를 입력한다. 반드시 본인의 키로 대체해야 한다.

In [23]:
# Infura HTTP 프로바이더 URL (본인의 키로 대체)
infura_url = "https://mainnet.infura.io/v3/1914acea63974a9284b713394080e9d3"

web3 인스턴스를 생성한다.

In [24]:
# Web3 인스턴스 생성
web3 = Web3(Web3.HTTPProvider(infura_url))

시작블록과 끝 블록을 지정한다.

In [28]:
# 확인 범위
start_block = 20000000
end_block = 20000050

트랜잭션 개수와 평균가스 비용을 저장할 리스트를 초기화해서 준비한다.

In [29]:
# 트랜잭션 개수와 평균 가스 비용을 저장할 리스트
tx_counts = []
total_fees = []
block_numbers = []

연결해서 값을 읽어온다.<BR>
각 블록의 트랜잭션 개수를 확인하고 총수수료를 계산 후 평균을 구하고 그래프로 그린다.

그래프로 그린다.

In [ ]:
# 연결 상태 확인
if web3.is_connected():
    print("Connected to Ethereum network")

    # 각 블록의 트랜잭션 개수와 총 수수료 계산
    for block_number in range(start_block, end_block + 1):
        block = web3.eth.get_block(block_number, full_transactions=True)
        tx_count = len(block['transactions'])
        tx_counts.append(tx_count)
        block_numbers.append(block_number)

        if tx_count > 0:
            total_gas_fee = 0
            for tx in block['transactions']:
                receipt = web3.eth.get_transaction_receipt(tx['hash'])
                gas_used = receipt['gasUsed']
                total_gas_fee += tx['gasPrice'] * gas_used

            total_fees.append(total_gas_fee / 1e18)  # ETH 단위로 변환
        else:
            total_fees.append(0)
#
        print(f"Block {block_number} has {tx_count} transactions with total fee {total_gas_fee / 1e18} ETH")

    # 총 트랜잭션 개수와 평균 수수료 계산
    total_tx_count = sum(tx_counts)
    overall_average_fee = sum(total_fees) / len(total_fees)
#    print(f"\nTotal transactions from block {start_block} to {end_block}: {total_tx_count}")
#    print(f"Overall average fee: {overall_average_fee:.6f} ETH")

    # 그래프 생성
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # 평균 수수료를 왼쪽 축에 표시
    ax1.set_xlabel('Block Number')
    ax1.set_ylabel('Average Fee (ETH)', color='blue')
    ax1.plot(block_numbers, total_fees, marker='o', linestyle='-', color='blue', label='Average Fee')
    ax1.tick_params(axis='y', labelcolor='blue')
    ax1.grid(True)

    # 평균 트랜잭션 수를 오른쪽 축에 표시
    ax2 = ax1.twinx()
    ax2.set_ylabel('Number of Transactions', color='green')
    ax2.plot(block_numbers, tx_counts, marker='o', linestyle='-', color='green', label='Number of Transactions')
    ax2.tick_params(axis='y', labelcolor='green')

    # 제목 및 범례 추가
    plt.title('Average Fee and Number of Transactions per Block')
    fig.tight_layout()
    plt.show()
else:
    print("Failed to connect to Ethereum network")
